# États des nappes phréatiques françaises

## Introduction
En 2023, le changement climatique provoque de plus en plus de sécheresse, il augmente par ailleurs les intensités des canicules et réduit les fréquences des phénomènes pluvieux. Ces phénomènes ont un impact sur les nappes phréatiques françaises. En effet, les nappes phréatiques sont des réservoirs d'eau souterrains qui se remplissent grâce aux précipitations et se vident grâce aux prélèvements. Les nappes phréatiques sont donc un indicateur de la sécheresse. L'objectif de ce projet est de déterminer l'état des nappes phréatiques françaises en 2023. Pour cela, nous allons utiliser les données de l'Agence de l'eau qui sont accessible en *open source*. Le but est d'analysé le niveau actuelle des nappes phréatiques et de le comparer avec les niveaux limites, ainsi nous pourrons déterminer l'état des nappes phréatiques en temps réels si les mesures sont faites régulièrement.

### Les mesures
Les mesures sont le calcul du niveau d'eau dans les nappes phréatiques, ces mesures peuvent être exprimé selon le NGF (nivellement général français) ou bien selon la profondeur relative du niveau d'eau par rapport au sol. Les mesures sont réalisée grâce un appareil de mesure appelée un piezomètre.

Les données sont disponibles sur le site ades.eaufrance.fr. Les nappes phréatiques sont identifié par un code BSS unique. Ce code est nécessaire pour toute les requêtes envoyées au serveur afin qu'il puisse identifié la nappe sur laquel nous souhaitons récupérer des informations.

Les données sont ensuite comparée à leur IPS (indicateur piézométrique standardisé) afin d'avoir des critères de niveau d'eau relatif à chaque nappe.


> *Les indicateurs piézométriques ont un rôle essentiel dans la gestion locale de la ressource en eau. Toutefois la diversité des critères utilisés pour leur détermination ne permet pas d’avoir une vision homogène de l’impact d’un déficit pluviométrique sur l’état des nappes libres à l’échelle nationale, contrairement à d'autres indicateurs. C’est ce qui a motivé la création de l'IPS (Indicateur Piézométrique Standardisé), qui permet d'avoir une vision homogène de l'état des nappes libres ou captives à l'échelle nationale.*

Voir [l'article sur le site du brgm](https://www.brgm.fr/fr/reference-projet-acheve/indicateur-piezometrique-standardise-ips-gestion-ressource-eau) pour plus d'information sur l'IPS.

# I. Importation des données chronique et en temps réel
Tout d'abord une requête est envoyé au serveur.
L'url est : **https://ades.eaufrance.fr/Fiche/MesureGraphSpliAnnee**

Les paramètres sont :
- Le code BSS : *obligatoire, Exemple : 07228X0017/PZF*
- Année : *obligatoire*
- DateMax : *obligatoie, Exemple : 01/01/2023*

In [31]:
import requests

UrlDonneeChroniqueTR = "https://ades.eaufrance.fr/Fiche/MesureGraphSpliAnnee?CodeBSS=07228X0017/PZF&Annees=2023&Couple=3&DateMax=15%2F06%2F2023"


On récupère la réponse du serveur

In [32]:
payload = {}
DonneeChroniqueTR = requests.request("GET", UrlDonneeChroniqueTR, headers="", data=payload)
print(DonneeChroniqueTR.text)

"{x : Date.UTC(2000,0,1,0,1), y :216},{x : Date.UTC(2000,1,1,0,1), y :215.94},{x : Date.UTC(2000,2,1,0,1), y :215.91},{x : Date.UTC(2000,3,1,0,1), y :215.92},{x : Date.UTC(2000,4,1,0,1), y :215.87},{x : Date.UTC(2000,5,1,0,1), y :215.83},{x : Date.UTC(2000,6,1,0,1), y : null },{x : Date.UTC(2000,7,1,0,1), y : null },{x : Date.UTC(2000,8,1,0,1), y : null },{x : Date.UTC(2000,9,1,0,1), y : null },{x : Date.UTC(2000,10,1,0,1), y : null },{x : Date.UTC(2000,11,1,0,1), y : null }"


## Traitement des données
Tout d'abord il faut transformer les données brut en format json

In [33]:
import json
JsonDonneeChroniqueTR = json.loads(DonneeChroniqueTR.text)
print(JsonDonneeChroniqueTR)

{x : Date.UTC(2000,0,1,0,1), y :216},{x : Date.UTC(2000,1,1,0,1), y :215.94},{x : Date.UTC(2000,2,1,0,1), y :215.91},{x : Date.UTC(2000,3,1,0,1), y :215.92},{x : Date.UTC(2000,4,1,0,1), y :215.87},{x : Date.UTC(2000,5,1,0,1), y :215.83},{x : Date.UTC(2000,6,1,0,1), y : null },{x : Date.UTC(2000,7,1,0,1), y : null },{x : Date.UTC(2000,8,1,0,1), y : null },{x : Date.UTC(2000,9,1,0,1), y : null },{x : Date.UTC(2000,10,1,0,1), y : null },{x : Date.UTC(2000,11,1,0,1), y : null }


On souhaite mettre les données sous forme d'un dictionnaire Mois-Mesure.
On commence par découper les données avec la fonction `split()`.

Ensuite on cast les valeurs de mesure en `float` avant de les rajouter dans le dictionnaire `Data`. À la fin du traitement de l'année, on met le dictionnaire `Data` dans le dictionnaire `finalData` à laquel on associe la clé de  l'année des mesures.

In [34]:
import re

# Séparation des mesures et suppressions des accolades fermantes
TabDonneeChroniqueTR_1 = JsonDonneeChroniqueTR.split("}")

# Initialisation des objets
# Initialisation du tableau des mois afin de les ajouté comme clé du dictionnaire Data.
mois = ["Janvier","Fevrier","Mars","Avril","Mai","Juin","Juillet","Aout","Septembre","Octobre", "Novembre","Decembre"]
Data = {}
finalData = {}


for i in range (len(TabDonneeChroniqueTR_1)) :
    # On passe au second niveau de regex, ici on enlève les accolades ouvrantes
    # L'expression de la première mesure commence par '{' et non pas par ',{' comme les autres, il faut donc ajouter une condition spéciale pour le premier élément.
    if TabDonneeChroniqueTR_1[i] == TabDonneeChroniqueTR_1[0]:
        # Grâce à l'expression Regex, on supprime l'accolade fermante du premier élément.
        DonneesTransformees = re.sub('[{]','', TabDonneeChroniqueTR_1[i])
    else:
        # On fait ceci pour chaque mesure donnée.
        DonneesTransformees = re.sub('[,]+[{]','', TabDonneeChroniqueTR_1[i])
    # Avec les opérations RegEx, le dernier élément est [''], on fait en sorte que ce résultat n'entre pas en compte dans les résultats finaux.
    if (DonneesTransformees!=''):
        # Séparation clé-valeur
        DonneesTransformees = DonneesTransformees.split(", ")
        DonneesTransformees_1 = DonneesTransformees[1].split(":")
        # On enlève les espaces et les blancs inutiles
        DonneesTransformees_1[1] = DonneesTransformees_1[1].replace(' ','')
        # Les mesures des mois ultérieurs à la date d'aujourd'hui sont "Null", s'il le sont, on remplace la chaine de caractère par la "valeur" None.
        if (DonneesTransformees_1[1] != "null"):
            Data [mois[i]] = float(DonneesTransformees_1[1])
        else :
            Data [mois[i]] = None

# On ajoute, ici pour l'exemple donc or boucle, les résultats dans le dictionnaire finalData avec la clé "2023"
finalData["2023"]=Data
print(finalData)

{'2023': {'Janvier': 216.0, 'Fevrier': 215.94, 'Mars': 215.91, 'Avril': 215.92, 'Mai': 215.87, 'Juin': 215.83, 'Juillet': None, 'Aout': None, 'Septembre': None, 'Octobre': None, 'Novembre': None, 'Decembre': None}}


# II. Importation des données IPS
Les données IPS sont les données lié aux critères permettant de classer les nappes phréatiques sur une échelle de 5 :
- Niveaux très hauts (Bleu marine)
- Niveaux hauts (Bleu France)
- Niveaux modérément hauts (Cyan)
- Niveaux autour de la moyenne (Vert)
- Niveaux modérément bas (jaune)
- Niveaux bas (orange)
- Niveaux très bas (rouge)

L'url est : **https://ades.eaufrance.fr/Fiche/MesureGraphSpliTabPeriode**

Les paramètres sont :
- Le code BSS : *obligatoire, Exemple : 07228X0017/PZF*
- La date de début : *obligatoire, Exemple : 01/01/2022*
- La date de fin : *obligatoire, Exemple : 01/01/2023*
- Le couple : *par défaut 3*

In [35]:
UrlDonneeIPS = "https://ades.eaufrance.fr/Fiche/MesureGraphSpliTabPeriode?CodeBSS=07228X0017/PZF&DateDebut=26%2F06%2F1992&DateFin=15%2F06%2F2023&Couple=3&_=1687102433152"

On récupère la réponse du serveur

In [36]:
payload_1 = {}
DonneeIPS = requests.request("GET", UrlDonneeIPS, headers="", data=payload_1)

## Traitement des données
De même que pour les données piezométriques, on transforme les données en json

In [37]:
JsonDonneeIPS = json.loads(DonneeIPS.text)

On transforme les données json en un dictionnaire Mois-Niveau

In [46]:
# Initialisation du dictionnaire
month_by_level = {}

# Les données JSON contiennent d'autre élément que les mesures, ici ces éléments sont répertoriés.
Not_Numerical_Data = ["DT_RowId","quantile","annee","Initial","Coche"]

# Pour chaque élément de l'élément "data" de JsonDonneIPS
for i in JsonDonneeIPS['data']:
    # On initialise un dictionnaire qui contiendra en clé le mois et en valeur le niveau de ce mois.
    LevelMonth = {}

    # i est un dictionnaire contenant notamment le mois et la valeur correspondante sour la forme {Mois:Janvier,Valeur:180.8}
    for x in i :
        # Ce qui nous interesse est seulement le nom du mois et sa valeur, toutes les clés contenu dans la liste Not_Numerical_Data ne sont pas considéré utile.
        if (x not in Not_Numerical_Data):
            # On ajoute au dictionnaire LevelMonth le couple Mois-Niveau
            LevelMonth[x] = float(i[x])
    # On ajoute au dictionnaire month_by_level Le couple Quantile(c.à.d le niveau)-{Mois-Niveau}
    month_by_level[i['quantile']]=LevelMonth
print (month_by_level)

{'Niveaux très haut': {'janvier': 216.92, 'fevrier': 217.36, 'mars': 217.34, 'avril': 216.87, 'mai': 216.75, 'juin': 216.68, 'juillet': 216.59, 'aout': 216.43, 'septembre': 216.42, 'octobre': 216.53, 'novembre': 216.5, 'decembre': 216.74}, 'Niveaux hauts': {'janvier': 216.59, 'fevrier': 216.62, 'mars': 216.62, 'avril': 216.53, 'mai': 216.58, 'juin': 216.51, 'juillet': 216.45, 'aout': 216.3, 'septembre': 216.3, 'octobre': 216.35, 'novembre': 216.33, 'decembre': 216.51}, 'Niveaux modérément hauts': {'janvier': 216.26, 'fevrier': 216.29, 'mars': 216.3, 'avril': 216.28, 'mai': 216.31, 'juin': 216.31, 'juillet': 216.25, 'aout': 216.12, 'septembre': 216.12, 'octobre': 216.13, 'novembre': 216.13, 'decembre': 216.19}, 'Niveaux autour de la moyenne': {'janvier': 216.18, 'fevrier': 216.21, 'mars': 216.22, 'avril': 216.21, 'mai': 216.22, 'juin': 216.22, 'juillet': 216.17, 'aout': 216.06, 'septembre': 216.04, 'octobre': 216.05, 'novembre': 216.06, 'decembre': 216.12}, 'Niveaux modérément bas': {'j

Une fois ceci fait, nous avons alors les données triées selon le niveau. Chaque niveau contient les 12 mois avec les valeurs limites de niveau qui leur sont associées.
Cependant, il serait plus intéressant que les données soient triées par mois, tel que chaque moi contienne les mesures limites des 5 niveaux. Cela permettrait de faire des comparaisons entre les mois sans devoir parcourir toutes les données.

In [39]:
# Initialisation du dictionnaire
level_by_month = {}

# On récupère la liste des mois
for i in month_by_level[list(month_by_level.keys())[0]]:
    # On initialise le dictionnaire qui contiendra les couples quantiles-mesures
    months = {}
    #
    i1 = i[0].upper()+i[1:]
    for x in month_by_level:
        months [x] = month_by_level[x][i]
    level_by_month[i1] = months
for i in level_by_month:
    print(i)
    # On inverse le dictionnaire, cette opération est nécessaire pour la suite, car nous allons comparer les valeurs du dictionnaire avec les valeurs IPS qui sont les limites supérieurs des niveaux d'eau.
    # C'est à dire que pour chaque donnée de niveau, la valeur IPS est la valeur supérieure, ce qui est par exemple entre les valeurs IPS du Niveau bas et du Niveau modérément bas est considéré comme Niveau modérément bas.
    level_by_month[i] = dict(reversed(list(level_by_month[i].items())))
    print(level_by_month[i])

Janvier
{'Niveaux très bas': 215.86, 'Niveaux bas': 215.96, 'Niveaux modérément bas': 216.1, 'Niveaux autour de la moyenne': 216.18, 'Niveaux modérément hauts': 216.26, 'Niveaux hauts': 216.59, 'Niveaux très haut': 216.92}
Fevrier
{'Niveaux très bas': 215.94, 'Niveaux bas': 216.02, 'Niveaux modérément bas': 216.15, 'Niveaux autour de la moyenne': 216.21, 'Niveaux modérément hauts': 216.29, 'Niveaux hauts': 216.62, 'Niveaux très haut': 217.36}
Mars
{'Niveaux très bas': 215.93, 'Niveaux bas': 216.02, 'Niveaux modérément bas': 216.15, 'Niveaux autour de la moyenne': 216.22, 'Niveaux modérément hauts': 216.3, 'Niveaux hauts': 216.62, 'Niveaux très haut': 217.34}
Avril
{'Niveaux très bas': 215.91, 'Niveaux bas': 216.0, 'Niveaux modérément bas': 216.14, 'Niveaux autour de la moyenne': 216.21, 'Niveaux modérément hauts': 216.28, 'Niveaux hauts': 216.53, 'Niveaux très haut': 216.87}
Mai
{'Niveaux très bas': 215.87, 'Niveaux bas': 215.98, 'Niveaux modérément bas': 216.14, 'Niveaux autour de la 

In [40]:
for x in finalData:
    # x = {'Janvier': 708.57, 'Février': 708.38, 'Mars': 708.58, 'Avril': 708.56, 'Mai': 708.43, 'Juin': 708.37, 'Juillet': None, 'Août': None, 'Septembre': None, 'Octobre': None, 'Novembre': None, 'Décembre': None}
    for i in finalData[x]:
        # i = 'Janvier'
        for y in level_by_month[i]:
            # y = {'Niveaux très haut': 708.44, 'Niveaux hauts': 708.39, 'Niveaux modérément hauts': 708.32, 'Niveaux autour de la moyenne': 708.29, 'Niveaux modérément bas': 708.27, 'Niveaux bas': 708.22, 'Niveaux très bas': 708.18}
            if (finalData[x][i] != None):
                if (finalData[x][i] > level_by_month[i][y]):
                    pass
                else:
                    print ("En ",x," au mois de ",i," le niveau est ",y)
                    break

En  2023  au mois de  Janvier  le niveau est  Niveaux modérément bas
En  2023  au mois de  Fevrier  le niveau est  Niveaux très bas
En  2023  au mois de  Mars  le niveau est  Niveaux très bas
En  2023  au mois de  Avril  le niveau est  Niveaux bas
En  2023  au mois de  Mai  le niveau est  Niveaux très bas
En  2023  au mois de  Juin  le niveau est  Niveaux très bas


# Utilisation de Plotly pour afficher les données en graphe
La libraire [plotly](https://plotly.com/python/) nous permet d'afficher beaucoup d'information graphiquement.
Nous devons d'abord transformer les données en matrices afin de les exploiter, ceci se fait grâce à la libraire [numpy](https://numpy.org)

# Évolution chronique du niveau de la nappe phréatique

In [118]:
import plotly.graph_objs as go
import numpy as np

fig = go.Figure()
fig.update_layout(
    title="Évolution chronique du niveau de la nappe phréatique",
    xaxis_title="Month",
    yaxis_title="Level",
    xaxis_type='category',
)
for x in finalData :
    fig.add_trace(go.Scatter(name=x,x=[i for i in finalData[x]],y=[finalData[x][i] for i in finalData[x]]))
fig.update()


for x in dict(reversed(list(month_by_level.items()))) :
    fig.add_trace(go.Scatter(name=x,x=[i[0].upper()+i[1:] for i in month_by_level[x]],y=[month_by_level[x][i] for i in month_by_level[x]], hoverinfo='x+y',fill='tonexty',mode='none'))
fig.update()
fig.show()